In [1]:
import json
import pandas as pd
import numpy as np
import string
import pickle
from xml.dom import minidom
import random
from tqdm import tqdm
import gzip
from gensim import utils

In [2]:
import re

In [3]:
# !pip3 install num2words

In [4]:
from num2words import num2words

In [5]:
import nltk

In [6]:
from googletrans import Translator

In [7]:
def preprocessing(text, lang):
    new_txt = []
    regex = re.compile('[^a-zA-Z ]')
    for t in text:
        try:
            t = re.sub(r"(\d+)", lambda x: num2words(int(x.group(0)), lang = lang), t)
            t = re.sub(r'\-', ' ', t)
            t = regex.sub(' ', t)
            t = re.sub(r'\s+', ' ', t)
            new_txt.append(t)
        except:
            t = t
    return new_txt

In [8]:
ms_data = []
with utils.open('mswiki-20201001-pages-articles-multistream.json.gz', 'rb') as f:
    for line in f:
        article = json.loads(line)
        for section_title, section_text in zip(article['section_titles'], article['section_texts']):
            txt = preprocessing(section_text.split('. ') , 'id')
            ms_data+= txt

In [9]:
pickle.dump( ms_data, open( "ms_data.p", "wb" ) )

In [9]:
grammar_ms = r"""
  NP: {<DET|ADJ|NOUN.*>+<DET|ADJ|NOUN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<ADP><NP>} 
      {<ADP><PROPN>}# Chunk prepositions followed by NP
  VP: {<VERB.*><NP|PP|CLAUSE|ADP>+$} # Chunk verbs and their arguments
      {<VERB*><NOUN*>}
      {<PART><VERB>}
  CLAUSE: {<NP><VP>}"""

In [10]:
grammar_en = r""" """

In [11]:
def get_phrase(dt, lgs):
    if lgs == 'ms':
        grammar = grammar_ms
    else:
        grammar= grammar_en    
    cp = nltk.RegexpParser(grammar)
    result = cp.parse(dt)
    max_deep = result.height()
    phrase = []
    for s in result.subtrees(lambda result: result.height() == 2):
        bb = []
        for dd in s.leaves():
            bb.append(dd[0])
        phrase.append(bb)
    if max_deep != 3:
        for s in result.subtrees(lambda result: result.height() == 3):
            bb = []
            for dd in s.leaves():
                bb.append(dd[0])
            phrase.append(bb)
    token = [d[0] for d in dt]
    idx = find_idx_word(phrase, token)
    return idx, phrase

In [13]:
import random

In [14]:
replace_ratio = [0.7,0.6,0.5]

In [15]:
tgt_lang = {'en': ['ms', 'ms','ms','ms', 'ms','zh-tw', 'zh-cn'], 'ms':['en','en', 'en', 'en', 'en', 'zh-tw', 'zh-cn']}

In [16]:
def find_idx_word(phrases, token):
    idx = []
    i = 0
    for ip, phrase in enumerate(phrases):
        idx_i = []
        for p in phrase:
            try :
                i = token.index(p, i+1)
            except:
                i = 0
                i = token.index(p, i)
            idx_i.append(i)
            idx_i = [idx_i[0], idx_i[len(idx_i)-1]]
        idx.append(idx_i)
    return idx

In [17]:
def replace_sent_with(data, text, index):
    del text[0]
    del index[0]
    
    #flagging
    for idx in index:
        for i in range(idx[0], idx[1]+1):
            data[i] = None
            
    # replace None with specific value
    for i, idx in enumerate(index):
        x = idx[0]
        data[x] = text[i]
        
    #remove None value
    clear_none = []
    for x in data:
        if x != None:
            clear_none.append(x)
            
    new_text = []
    for sublist in clear_none:
        if type(sublist) is list:
            for item in sublist:
                new_text.append(item)
        else:
            new_text.append(sublist)
        
    return new_text

In [18]:
translator = Translator()

In [19]:
import time

In [ ]:
# new_ms_data = []
# del model

In [34]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import malaya
model = malaya.pos.transformer(model = 'albert')
# tokens = model.predict(sentence)

In [21]:
new_ms_data = pickle.load( open( "new_ms_phrase_augment.p", "rb" ) )

In [130]:
for num_dt, dt in enumerate(tqdm(ms_data[len(new_ms_data):])):
    print(dt)
    if random.random() < replace_ratio[0]:
        try:
            token = model.predict(dt)
        except:
            continue
        idx, txt = get_phrase(token, 'ms')
        dt = [d[0] for d in token]
        translate_txt = [['']]
        translate_idx = [[0,0]]
        for j in range(round(replace_ratio[1]*len(txt))):
            i = random.randint(0, len(txt)-1)
            if idx[i] not in translate_idx:
                lang = random.choice(tgt_lang['ms'])
                txt_trans = translator.translate((' '.join(txt[i])), dest=lang, src='ms')
                translate_txt.append(txt_trans.text.split())
                translate_idx.append(idx[i])
        dt = replace_sent_with(dt, translate_txt, translate_idx)
        new_ms_data.append(' '.join(dt))
    else:
        if random.random() < replace_ratio[2]:
            try:
                dt = translator.translate(dt, dest=random.choice(tgt_lang['ms']), src='ms').text
            except:
                dt = dt
        new_ms_data.append(dt)
    print(dt)
    break
    if (num_dt+1) % 100 == 0:
        print('sleep')
        time.sleep(300)

  0%|          | 0/2129567 [00:00<?, ?it/s]

Iklim di Laut Carribean dipengaruhi oleh Arus Humbolt dari utara


  0%|          | 0/2129567 [00:00<?, ?it/s]

['Iklim', '在海上', 'Carribean', 'dipengaruhi', 'oleh', 'Arus', 'Humbolt', 'dari', 'utara']


In [111]:
' '.join(['Along', 'the', 'coast', 'terdapat', 'teluk', 'dan', 'tanjung', 'yang', 'berpontensi', 'dimajukan', 'sebagai', 'tourist', 'areas'])

'Along the coast terdapat teluk dan tanjung yang berpontensi dimajukan sebagai tourist areas'

In [135]:
pickle.dump( new_ms_data, open( "new_ms_phrase_augment.p", "wb" ) )

In [138]:
a = pickle.load( open( "new_ms_phrase_augment.p", "rb" ) )

In [142]:
len(a)

58504